In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pneumo import dataset
from torch.utils.data import DataLoader
import os
import glob
import albumentations as A

In [3]:
IMG_PATH = "/home/wilmer-linux/Projects/data/pneumo_s1/dicom-images-train/"
LABEL_PATH = os.path.join(IMG_PATH, '../train-rle.csv')

fns = glob.glob(os.path.join(IMG_PATH, "**/*.dcm"), recursive=True)

In [4]:
from sklearn.model_selection import train_test_split

train_fns, valid_fns = train_test_split(fns, train_size=0.9, random_state=42, shuffle=True)
print(len(train_fns), len(valid_fns))

10880 1209


In [5]:
from albumentations import (
    Compose, HorizontalFlip,
    RandomBrightness, RandomContrast, RandomGamma,OneOf,
    ShiftScaleRotate,GridDistortion, ElasticTransform, JpegCompression, HueSaturationValue,
    RandomBrightness, RandomContrast, Blur, MotionBlur, MedianBlur, GaussNoise,CenterCrop,
    IAAAdditiveGaussianNoise,GaussNoise,OpticalDistortion,RandomSizedCrop, Resize, RandomBrightnessContrast, ToFloat
)

IMG_SIZE= (256, 256)

TRAIN_AUGS = Compose([
    ToFloat(max_value=1),
    Resize(IMG_SIZE[0], IMG_SIZE[1]),
    HorizontalFlip(p=0.5),
    ShiftScaleRotate(p=0.2),
    OneOf([
        RandomGamma(),
        RandomBrightnessContrast(),
         ], p=0.2),
    OneOf([
        ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        GridDistortion(),
        OpticalDistortion(distort_limit=2, shift_limit=0.5),
        ], p=0.2),
],p=1)

VALID_AUGS = Compose([
    ToFloat(max_value=1),
    Resize(IMG_SIZE[0], IMG_SIZE[1])
])

In [6]:
bs = 4

train_ds = dataset.PneumoDataset(LABEL_PATH, train_fns, transforms=TRAIN_AUGS)
valid_ds = dataset.PneumoDataset(LABEL_PATH, valid_fns, transforms=VALID_AUGS)

train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=2, multiprocessing_context='spawn')
valid_loader = DataLoader(valid_ds, batch_size=bs, shuffle=False, num_workers=0)

In [7]:
from pneumo.models import UNetLightning
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger

unet = UNetLightning(learning_rate=1e-3).cuda()

wandb_logger = WandbLogger()
es = EarlyStopping(monitor="train_loss",min_delta=0.01, patience=3)

trainer = Trainer(
    gpus=1,
    auto_select_gpus=True, 
    amp_backend="apex", amp_level="02", 
    logger=wandb_logger, 
    callbacks=[es]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
ckpt_path = "/home/wilmer-linux/Projects/code/pneumo/ckpt/"

trainer.fit(unet, train_dataloader=train_loader, val_dataloaders=valid_loader)

/home/wilmer-linux/miniconda3/envs/faceai-nada/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:730: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: Currently logged in as: wilmer (use `wandb login --relogin` to force relogin)



  | Name              | Type                | Params
----------------------------------------------------------
0 | encoder           | EfficientNetEncoder | 10.7 M
1 | decoder           | UnetDecoder         | 2.5 M 
2 | segmentation_head | Sequential          | 1.2 K 
----------------------------------------------------------
13.2 M    Trainable params
0         Non-trainable params
13.2 M    Total params
52.638    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/wilmer-linux/miniconda3/envs/faceai-nada/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/wilmer-linux/miniconda3/envs/faceai-nada/lib/python3.9/site-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/wilmer-linux/miniconda3/envs/faceai-nada/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machin

Training: 0it [00:00, ?it/s]